In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import datetime
import json
i = 1
# Get the current year
year = datetime.datetime.now().year

sous_sous_formations_uniques = set()
url_formations = 'https://www.cegos.fr/formations'
headers = {
    'User-Agent': 'Opera_gx'
}

response = requests.get(url_formations, headers=headers)

if response.ok:
    soup = BeautifulSoup(response.text, 'lxml')

    formations_dict = {}  # Dictionnaire pour stocker les formations et leurs informations

    lis = soup.find_all('li', class_='city__list-item')
    for li in lis:
        les_a = li.find_all('a')
        for a in les_a:
            if 'href' in a.attrs:
                href = a['href']
                if href.startswith('/formations/'):

                    full_link = f'https://www.cegos.fr{href}'  # Construire le lien complet
                    formation = a.text.strip()  # Nettoyer la chaîne en supprimant les espaces superflus

                    # Créez un dictionnaire pour chaque formation avec des informations supplémentaires
                    formation_info = {
                        'Lien': full_link,
                    }

                    if formation not in formations_dict:
                        formations_dict[formation] = {}

                    formations_dict[formation]['info'] = formation_info  # Associez les informations de la formation

    # Parcourez le dictionnaire des formations
    for formation, info in formations_dict.items():
        url_sous_formations = info['info']['Lien']
        headers = {
            'User-Agent': 'Opera_gx'
        }

        response = requests.get(url_sous_formations, headers=headers)

        if response.ok:
            soup = BeautifulSoup(response.text, 'lxml')

            # Supposons que vous voulez filtrer les éléments par attribut data-year
            year = str(year)  # Remplacez par la valeur que vous recherchez

            lus = soup.find_all('ul', class_='domaine__formations-items')  # Utilisez 'ul' au lieu de 'lu'
            for lu in lus:
                lis = lu.find_all('li', {'data-year': year})
                for li in lis:
                    h3 = li.find('h3', class_='u-myl')
                    sous_formation = h3.find('strong').text

#                    print(sous_formation)

                    a_sous_sous_formation = li.find_all('a', class_='domaine__formation-link')

                    sous_formations_dict = {}
                    #Les noms de sous_sous_formations incluent le nombre de jours et d'autres informations donc on extrait que le nom.
#=========================================================

                    for a in a_sous_sous_formation:

                        sous_sous_formation1 = a.find('div', class_='domaine__formation-title')
                        sous_sous_formation = sous_sous_formation1.text.strip()
#=========================================================
                        # Initialisation de l'ensemble pour stocker les identifiants uniques des sous-sous-formations
                        if sous_sous_formation not in sous_sous_formations_uniques:
                          sous_sous_formation_info = {
                              'Lien': f'https://www.cegos.fr{a["href"]}',
                              'Prix': 0,
                              'Duree' : 0,
                              'Description' : '',
                              'note' : '',
                              'nombre_avis' : '',
                              'niveau' : '',
                              'Pour Qui': ''
                          }
                          sous_formations_dict[sous_sous_formation] = sous_sous_formation_info
                          sous_sous_formations_uniques.add(sous_sous_formation)
                    info[sous_formation] = sous_formations_dict
# Convertir le dictionnaire en JSON
json_data = json.dumps(formations_dict, ensure_ascii=False, indent=4)

# Nom du fichier de sortie
output_filename = 'formations.json'

# Écrire le JSON dans le fichier
with open(output_filename, 'w', encoding='utf-8') as json_file:
    json_file.write(json_data)

print(f'Données JSON écrites dans {output_filename}')


In [ ]:
formations_dict['Droit des affaires']['Formation juridique - Droit des sociétés']

In [1]:
from bs4 import BeautifulSoup
import requests
import re

def extraire_informations_formation(url):
    headers = {
        'User-Agent': 'Opera_gx'
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'lxml')

        note_text = None  # Initialisation de la variable note_text à None
        avis_text = None
                # Initialisation des variables à None
        note_text = None
        avis_text = None
        price = None
        duration_str = None
        description = None
        level = None
        Pour_Qui = None

        # Recherche de tous les éléments <a> avec la classe "product__rating"
        rating_links = soup.find_all('a', class_='product__rating')

        for rating_link in rating_links:
            # Vérifie si l'élément <a> contient la classe "product__rating"
            if 'product__rating' in rating_link['class']:
                # Recherche de l'élément avec la classe 'rating__note' à l'intérieur de cet élément
                rating_note = rating_link.find('div', class_='rating__note')
                if rating_note:
                    note_text = rating_note.get_text()

                    # Recherche de l'élément avec la classe 'rating__count' pour obtenir le nombre d'avis
                    rating_count = rating_link.find('div', class_='rating__count')
                    if rating_count:
                        avis_text = rating_count.get_text().strip()
                    else:
                        avis_text = None

        script = soup.find("script", string=re.compile("let mainProduct = {.*};", re.DOTALL))

        if script:
            match_price = re.search(r'"price_without_vat":"(.*?)"', script.string)
            match_duration_days = re.search(r'"duration_classroom_days":\s*"*(\d+\.?\d*)', script.string, re.I)
            match_duration_min = re.search(r'"duration_classroom_min":(\d+)', script.string)

            price = match_price.group(1) if match_price else None

            if match_duration_days and match_duration_min:
                # Convertir la durée en nombre à virgule flottante
                duration_days = float(match_duration_days.group(1))
                # Convertir la durée en un entier
                duration_days = int(duration_days)

                duration_min = int(match_duration_min.group(1))
                # Convertir la durée en heures et minutes
                hours = duration_min // 60
                minutes = duration_min % 60

                # Créer la chaîne de durée au format 'x jours (y heures)'
                duration_str = f'{duration_days} jours ({hours} heures)'
            else:
                duration_str = None

        # Recherchez la balise <meta> avec l'attribut name="Description"
        meta_description = soup.find("meta", attrs={"name": "Description"})

        # Obtenez le contenu de l'attribut "content"
        description = meta_description.get("content") if meta_description else None

        #----------------------------

        level_search = soup.find('div',class_ = "product__for product__for--required")

        # Vérifier si le contenu de la liste contient le mot 'Aucun'
        if '"Aucun."' in level_search.get_text():
            level = "débutant"
        else:
          level = "non débutant"


        Pour_Qui_search = soup.find('div',class_ = "product__for product__for--whom")

        # Extraire tous les éléments li dans une liste Python
        Pour_Qui = [li.get_text().replace('::marker', '').replace('"', '').replace('.', '').replace('\xa0', ' ').strip() for li in Pour_Qui_search.find_all('li')]
        # Pour remplacer les espaces insécables par des espaces standards

        return note_text, avis_text, price, duration_str, description, level, Pour_Qui

    return None, None, None, None, None, None


In [17]:
input_filename = 'formations.json'

with open(input_filename, 'r', encoding='utf-8') as json_file:
    formations_dict = json.load(json_file)

# Parcourir et mettre à jour les sous-champs "Prix" pour les 5 premières formations
count = 0
for formation, formation_info in formations_dict.items():
    for sous_formation, sous_formation_info in formation_info.items():
        if count < 200 and isinstance(sous_formation_info, dict):
            for sous_sous_formation, sous_sous_formation_info in sous_formation_info.items():
                if count < 200 and isinstance(sous_sous_formation_info, dict):
                    link_sous_sous_formation = sous_sous_formation_info['Lien']
                    note, avis, price, duration, description, level, Pour_Qui = extraire_informations_formation(link_sous_sous_formation)

                    sous_sous_formation_info['Duree'] = duration
                    sous_sous_formation_info['Prix'] = price
                    sous_sous_formation_info['Description'] = description
                    sous_sous_formation_info['note'] = note
                    sous_sous_formation_info['nombre_avis'] = avis
                    sous_sous_formation_info['niveau'] = level
                    sous_sous_formation_info['Pour Qui'] = Pour_Qui
                    count += 1

# Une fois que vous avez effectué toutes les modifications nécessaires, vous pouvez sauvegarder les données mises à jour dans le fichier JSON
with open(input_filename, 'w', encoding='utf-8') as json_file:
    json.dump(formations_dict, json_file, ensure_ascii=False, indent=4)

print(f'Les 20 formations ont été mis à jour dans {input_filename}')

Les 20 formations ont été mis à jour dans formations.json


In [18]:
pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 2.0 MB/s eta 0:00:00


In [3]:
import nltk
from nltk.corpus import stopwords
import unidecode
import re

# Télécharger les stop words (à faire une fois)
# nltk.download('stopwords')

def clean_course_name(course_name):
    # Votre liste initiale de mots à exclure
    custom_exclude_words = {
        'et', 'des', 'les', 'de', 'la', 'le', 'en', 'du', 'pour', 'comment',
        'avec', 'sans', 'sous', 'par', 'dans', 'sur', 'entre', 'contre', 'vers',
        'après', 'avant', 'chez', 'pendant', 'depuis', 'jusque', 'jusqu', 'tandis',
        'que', 'comme', 'si', 'lorsque', 'puisque', 'quoique', 'bien', 'ainsi'
    }

    # Stop words en français et en anglais
    stop_words_fr = set(stopwords.words('french'))
    stop_words_en = set(stopwords.words('english'))
    exclude_words = custom_exclude_words.union(stop_words_fr, stop_words_en)

    # Remplacer les caractères accentués
    course_name = unidecode.unidecode(course_name)

    # Supprimer le caractère avant les guillemets doubles
    course_name = re.sub(r".''", '', course_name)

    # Convertir en minuscules, supprimer les caractères spéciaux (sauf les tirets, les barres obliques)
    course_name = re.sub(r'[^A-Za-z\s/-]', '', course_name.lower())

    # Split sur les espaces, les tirets, et les barres obliques
    words = re.split(r'[\s/-]+', course_name)

    # Filtrer les mots, les mettre au singulier si nécessaire et exclure les mots d'une lettre
    filtered_words = [word for word in words if word not in exclude_words and len(word) > 1]

    return " ".join(filtered_words)


# Exemple d'utilisation
course_name = "Commercial - Ventes"
cleaned_name = clean_course_name(course_name)
print(cleaned_name)  # Devrait afficher une liste de mots clés nettoyés

commercial, ventes


In [21]:
python -m spacy download fr_core_news_sm

SyntaxError: ignored

In [5]:
import spacy
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop

# Charger le modèle français
nlp = spacy.load("fr_core_news_sm")

def filter_keywords_spacy(sentence):
    # Tokenize la phrase avec Spacy
    doc = nlp(sentence)


    # Filtrer les stop words et les mots non significatifs, sauf pour les exceptions
    keywords = [token.text for token in doc if (token.text.lower() not in fr_stop and token.pos_ in ["ADJ","NOUN", "VERB", "PROPN"])]

    return ', '.join(keywords)


INSERT SCRIPT

In [10]:
import csv

# Noms des fichiers
input_json_filename = "C:/Users/aanis/Desktop/P03/Données/CEGOS/formations_CEGOS.json"  # Nom de votre fichier JSON
output_sql_formation_filename = 'insert_formation.sql'
output_sql_sous_formation_filename = 'insert_sous_formation.sql'
output_sql_sous_sous_formation_filename = 'insert_sous_sous_formation.sql'
output_csv_formation_filename = 'insert_formation.csv'
output_csv_sous_formation_filename = 'insert_sous_formation.csv'
output_csv_sous_sous_formation_filename = 'insert_sous_sous_formation.csv'

# Ouvrir le fichier JSON pour lecture
with open(input_json_filename, 'r', encoding='utf-8') as json_file:
    formations_dict = json.load(json_file)

# Ouvrir les fichiers SQL et CSV pour écriture
with open(output_sql_formation_filename, 'w', encoding='utf-8') as sql_formation_file, \
     open(output_sql_sous_formation_filename, 'w', encoding='utf-8') as sql_sous_formation_file, \
     open(output_sql_sous_sous_formation_filename, 'w', encoding='utf-8') as sql_sous_sous_formation_file, \
     open(output_csv_formation_filename, 'w', newline='', encoding='utf-8') as csv_formation_file, \
     open(output_csv_sous_formation_filename, 'w', newline='', encoding='utf-8') as csv_sous_formation_file, \
     open(output_csv_sous_sous_formation_filename, 'w', newline='', encoding='utf-8') as csv_sous_sous_formation_file:


    csv_formation_writer = csv.writer(csv_formation_file)
    csv_sous_formation_writer = csv.writer(csv_sous_formation_file)
    csv_sous_sous_formation_writer = csv.writer(csv_sous_sous_formation_file)

    # Écrire les en-têtes pour chaque fichier CSV
    csv_formation_writer.writerow(['RefDomaine', 'DomaineDeFormation','MotsCles'])
    csv_sous_formation_writer.writerow(['RefSousDomaine', 'RefDomaine', 'SousDomaineDeFormation','MotsCles'])
    csv_sous_sous_formation_writer.writerow(['RefSOUSSOUSDomaineF', 'RefSOUSDomaineF', 'Le_nom', 'Description', 'Notes', 'Nombre_avis', 'Duree', 'Niveau', 'Liens','MotsCles'])

    numéro_formation = 1


    for formation, formation_info in formations_dict.items():
        numéro_sous_formation = 1
        # Ref and corrected formation name
        ref_domaine = f"CEGOS_DomaineF-{numéro_formation:02}"
        formation_corrigée = formation.replace("'", "''")

        mots_clefs_formation1 = clean_course_name(formation_corrigée)
        mots_clefs_formation = filter_keywords_spacy(mots_clefs_formation1)

        # SQL Insert
        sql_formation_insert = f"INSERT INTO DomaineFormation (RefDomaine, DomaineDeFormation,MotsCles) VALUES ('{ref_domaine}', '{formation_corrigée}','{mots_clefs_formation}');\n"
        sql_formation_file.write(sql_formation_insert)

        # CSV Insert
        csv_formation_writer.writerow([ref_domaine, formation_corrigée,mots_clefs_formation])

        for sous_formation, sous_formation_info in formation_info.items():

            numéro_sous_sous_formation = 1
            if sous_formation == 'info':
                continue

            # Ref and corrected sub-formation name
            ref_sous_domaine = ref_domaine + f"-{numéro_sous_formation:02}"
            sous_formation_corrigée = sous_formation.replace("'", "''")

            mots_clefs_sous_formation1 = clean_course_name(sous_formation_corrigée)
            mots_clefs_sous_formation = filter_keywords_spacy(mots_clefs_sous_formation1)
            # SQL Insert
            sql_sous_formation_insert = f"INSERT INTO SousDomaineFormation (RefSousDomaine, RefDomaine, SousDomaineDeFormation, MotsCles) VALUES ('{ref_sous_domaine}', '{ref_domaine}', '{sous_formation_corrigée}','{mots_clefs_sous_formation}');\n"
            sql_sous_formation_file.write(sql_sous_formation_insert)

            # CSV Insert
            csv_sous_formation_writer.writerow([ref_sous_domaine, ref_domaine, sous_formation_corrigée,mots_clefs_sous_formation])

            numéro_sous_formation += 1

            for sous_sous_formation, sous_sous_formation_info in sous_formation_info.items():
                ref_sous_sous_domaine = ref_sous_domaine + f"-{numéro_sous_sous_formation:02}"
                sous_sous_formation_corrigé = sous_sous_formation.replace("'", "''")

                mots_clefs_sous_sous_formation1 = clean_course_name(sous_sous_formation_corrigé)
                mots_clefs_sous_sous_formation = filter_keywords_spacy(mots_clefs_sous_sous_formation1)

                # Replace apostrophes in each string value
                for key in sous_sous_formation_info:
                    if isinstance(sous_sous_formation_info[key], str):
                        sous_sous_formation_info[key] = sous_sous_formation_info[key].replace("'", "''")

                # SQL Insert
                sql_sous_sous_formation_insert = (
                    "INSERT INTO SOUSSOUSDomaineFormation "
                    "(RefSOUSSOUSDomaineF, RefSOUSDomaineF, Le_nom, Description, Notes, Nombre_avis, Duree, Niveau, Liens, MotsCles) VALUES "
                    f"('{ref_sous_sous_domaine}', '{ref_sous_domaine}', '{sous_sous_formation_corrigé}', '{sous_sous_formation_info['Description']}', '{sous_sous_formation_info['note']}', '{sous_sous_formation_info['nombre_avis']}', '{sous_sous_formation_info['Duree']}', '{sous_sous_formation_info['niveau']}', '{sous_sous_formation_info['Lien']}','{mots_clefs_sous_sous_formation}');\n"
                )
                sql_sous_sous_formation_file.write(sql_sous_sous_formation_insert)

                # CSV Insert
                csv_sous_sous_formation_writer.writerow([ref_sous_sous_domaine, ref_sous_domaine, sous_sous_formation_corrigé, sous_sous_formation_info['Description'], sous_sous_formation_info['note'], sous_sous_formation_info['nombre_avis'], sous_sous_formation_info['Duree'], sous_sous_formation_info['niveau'], sous_sous_formation_info['Lien'],mots_clefs_sous_sous_formation])

                numéro_sous_sous_formation += 1

        numéro_formation += 1


In [ ]:
import nltk
from nltk.corpus import stopwords
import re
from unidecode import unidecode

# Télécharger les stop words (à faire une fois)
nltk.download('stopwords')

def clean_course_name_coursera(course_name):
    # Votre liste initiale de mots à exclure
    custom_exclude_words = {
        'et', 'des', 'les', 'de', 'la', 'le', 'en', 'du', 'pour', 'comment',
        'avec', 'sans', 'sous', 'par', 'dans', 'sur', 'entre', 'contre', 'vers',
        'après', 'avant', 'chez', 'pendant', 'depuis', 'jusque', 'jusqu', 'tandis',
        'que', 'comme', 'si', 'lorsque', 'puisque', 'quoique', 'bien', 'ainsi'
    }

    # Stop words en français et en anglais
    stop_words_fr = set(stopwords.words('french'))
    stop_words_en = set(stopwords.words('english'))
    exclude_words = custom_exclude_words.union(stop_words_fr, stop_words_en)

    # Remplacer les caractères accentués
    course_name = unidecode(course_name)


    # Supprimer le caractère avant les guillemets doubles
    course_name = re.sub(r".''", '', course_name)

    # Convertir en minuscules, supprimer les caractères spéciaux (sauf les tirets, les barres obliques)
    course_name = re.sub(r'[^A-Za-z\s/-]', '', course_name.lower())

    # Split sur les espaces, les tirets, et les barres obliques
    words = re.split(r'[\s/-]+', course_name)

    # Filtrer les mots, les mettre au singulier si nécessaire et exclure les mots d'une lettre
    filtered_words = [word for word in words if word not in exclude_words and len(word) > 1]

    return ", ".join(filtered_words)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
import csv
import json


# Define your JSON file name
input_json_filename = 'Coursera.json'

# Output file names
output_sql_formation_filename = 'Coursera_insert_formation.sql'
output_sql_sous_formation_filename = 'Coursera_insert_sous_formation.sql'
output_sql_sous_sous_formation_filename = 'Coursera_nsert_sous_sous_formation.sql'
output_csv_formation_filename = 'Coursera_insert_formation.csv'
output_csv_sous_formation_filename = 'Coursera_insert_sous_formation.csv'
output_csv_sous_sous_formation_filename = 'Coursera_insert_sous_sous_formation.csv'

# Read JSON file
with open(input_json_filename, 'r', encoding='utf-8') as json_file:
    formations_dict = json.load(json_file)

# Open SQL and CSV files for writing
with open(output_sql_formation_filename, 'w', encoding='utf-8') as sql_formation_file, \
     open(output_sql_sous_formation_filename, 'w', encoding='utf-8') as sql_sous_formation_file, \
     open(output_sql_sous_sous_formation_filename, 'w', encoding='utf-8') as sql_sous_sous_formation_file, \
     open(output_csv_formation_filename, 'w', newline='', encoding='utf-8') as csv_formation_file, \
     open(output_csv_sous_formation_filename, 'w', newline='', encoding='utf-8') as csv_sous_formation_file, \
     open(output_csv_sous_sous_formation_filename, 'w', newline='', encoding='utf-8') as csv_sous_sous_formation_file:

    # CSV Writers
    csv_formation_writer = csv.writer(csv_formation_file)
    csv_sous_formation_writer = csv.writer(csv_sous_formation_file)
    csv_sous_sous_formation_writer = csv.writer(csv_sous_sous_formation_file)

    # Write headers for each CSV file
    csv_formation_writer.writerow(['RefDomaine', 'DomaineDeFormation', 'MotsCles'])
    csv_sous_formation_writer.writerow(['RefSousDomaine', 'RefDomaine', 'SousDomaineDeFormation', 'MotsCles'])
    csv_sous_sous_formation_writer.writerow(['RefSOUSSOUSDomaineF', 'RefSOUSDomaineF', 'Le_nom', 'Description', 'Notes', 'Nombre_avis', 'Duree', 'Niveau', 'Liens', 'MotsCles'])

    numero_formation = 1

    for formation, sous_domaines in formations_dict.items():
        ref_domaine = f"COURSERA_DomaineF-{numero_formation:02}"
        formation_corrigee = formation.replace("'", "''")
        mots_clefs_formation = clean_course_name_coursera(formation_corrigee)

        # SQL and CSV for formation
        sql_formation_insert = f"INSERT INTO DomaineFormation (RefDomaine, DomaineDeFormation, MotsCles) VALUES ('{ref_domaine}', '{formation_corrigee}', '{mots_clefs_formation}');\n"
        sql_formation_file.write(sql_formation_insert)
        csv_formation_writer.writerow([ref_domaine, formation_corrigee, mots_clefs_formation])

        numero_sous_formation = 1
        for sous_formation, cours in sous_domaines.items():
            ref_sous_domaine = f"{ref_domaine}-{numero_sous_formation:02}"
            sous_formation_corrigee = sous_formation.replace("'", "''")
            mots_clefs_sous_formation = clean_course_name_coursera(sous_formation_corrigee)

            # SQL and CSV for sous_formation
            sql_sous_formation_insert = f"INSERT INTO SousDomaineFormation (RefSousDomaine, RefDomaine, SousDomaineDeFormation, MotsCles) VALUES ('{ref_sous_domaine}', '{ref_domaine}', '{sous_formation_corrigee}', '{mots_clefs_sous_formation}');\n"
            sql_sous_formation_file.write(sql_sous_formation_insert)
            csv_sous_formation_writer.writerow([ref_sous_domaine, ref_domaine, sous_formation_corrigee, mots_clefs_sous_formation])

            numero_sous_sous_formation = 1
            for cours_nom, cours_info in cours.items():
                ref_sous_sous_domaine = f"{ref_sous_domaine}-{numero_sous_sous_formation:02}"
                cours_nom_corrigé = cours_nom.replace("'", "''")
                mots_clefs_cours = clean_course_name_coursera(cours_nom_corrigé)

                # Prepare data for SQL and CSV
                description = cours_info.get('Course_description', '').replace("'", "''")
                notes = cours_info.get('stars', '')
                nombre_avis = cours_info.get('Number_of_Reviews', '').replace('avis', '').strip()
                duree = cours_info.get('Duration', '')
                niveau = cours_info.get('Difficulty', '')
                liens = cours_info.get('Courses URLs', '')

                # SQL and CSV for sous_sous_formation
                sql_sous_sous_formation_insert = f"INSERT INTO SOUSSOUSDomaineFormation (RefSOUSSOUSDomaineF, RefSOUSDomaineF, Le_nom, Description, Notes, Nombre_avis, Duree, Niveau, Liens, MotsCles) VALUES ('{ref_sous_sous_domaine}', '{ref_sous_domaine}', '{cours_nom_corrigé}', '{description}', '{notes}', '{nombre_avis}', '{duree}', '{niveau}', '{liens}', '{mots_clefs_cours}');\n"
                sql_sous_sous_formation_file.write(sql_sous_sous_formation_insert)
                csv_sous_sous_formation_writer.writerow([ref_sous_sous_domaine, ref_sous_domaine, cours_nom_corrigé, description, notes, nombre_avis, duree, niveau, liens, mots_clefs_cours])

                numero_sous_sous_formation += 1

            numero_sous_formation += 1

        numero_formation += 1


FileNotFoundError: [Errno 2] No such file or directory: 'Coursera.json'